In [1]:
import pandas as pd

In [44]:
NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]

peaks = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/k562_max_intersect_atac_dnase.bed", sep="\t", names=NARROWPEAK_SCHEMA)

In [45]:
#peaks=peaks.sample(10000).reset_index()
peaks.head()

,chr,start,end,1,2,3,4,5,6,summit
0,chr1,267975,268142,Peak_206400,259,.,3.37837,25.95865,24.08834,81
1,chr1,778339,779268,Peak_104815,901,.,3.81137,90.10805,87.93501,775
2,chr1,778339,779268,Peak_1380,1000,.,30.05122,2807.95850,2803.66772,469
3,chr1,869601,870290,Peak_113087,786,.,5.86433,78.67642,76.53883,569
4,chr1,869601,870290,Peak_25869,1000,.,20.60247,604.90887,602.05859,318


In [46]:
def get_cts(peaks_df, bw, width):
    """
    Fetches values from a bigwig bw, given a df with minimally
    chr, start and summit columns. Summit is relative to start.
    Retrieves values of specified width centered at summit.
    "cts" = per base counts across a region
    """
    vals = []
    for i, r in peaks_df.iterrows():
        vals.append(np.nan_to_num(bw.values(r['chr'], 
                                            r['start'] + r['summit'] - width//2,
                                            r['start'] + r['summit'] + width//2)))
        
    return np.array(vals)

In [47]:
import pyBigWig as pw
import numpy as np
from scipy.spatial.distance import jensenshannon


In [48]:
dnase_counts=pw.open("/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/DNASE_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/merged.K562.counts.bw")
dnase_profile=pw.open("/mnt/lab_data3/anusri/chrombpnet/results/chrombpnet/DNASE_PE/K562/uncorrected_model_08.22.2022_filters_512_dil_8/merged.K562.profile.bw")


In [49]:
def jsd_pred(profile1_preds, profile2_preds, peaks_df_1):
    assert(profile1_preds.shape==profile2_preds.shape)
    jsd_vals=[]
    regions = []
    for key in range(profile1_preds.shape[0]):
        jsd_val = jensenshannon(np.abs(profile1_preds[key]), np.abs(profile2_preds[key]))
        if np.isnan(jsd_val):
            #print(np.sum(profile1_preds[key]),np.sum(profile2_preds[key]))
            pass
        else:
            jsd_vals.append(jsd_val)
            regions.append(peaks_df_1.loc[key,"chr"]+":"+str(peaks_df_1.loc[key,"start"]+peaks_df_1.loc[key,"summit"]-500)+":"+str(peaks_df_1.loc[key,"start"]+peaks_df_1.loc[key,"summit"]+500))
    
    return [np.mean(jsd_vals), np.std(jsd_vals)], jsd_vals, regions


In [50]:
dnase_counts_d = get_cts(peaks, dnase_counts, 1000)
dnase_profile_d = get_cts(peaks, dnase_profile, 1000)

In [51]:
vals = jsd_pred(dnase_counts_d, dnase_profile_d, peaks)

In [52]:
np.array(vals[1])[np.argsort(vals[1], )][::-1][0:100]

array([0.52968094, 0.52270888, 0.52192487, 0.52192487, 0.51228953,
       0.51228953, 0.50581285, 0.49744543, 0.48999625, 0.48999625,
       0.4892894 , 0.4892894 , 0.4892894 , 0.48917244, 0.48772878,
       0.48724715, 0.48724715, 0.48527424, 0.48527424, 0.48441455,
       0.48383587, 0.48379064, 0.48379064, 0.48379064, 0.48375649,
       0.48360029, 0.483547  , 0.48349962, 0.48308974, 0.48308974,
       0.48263751, 0.48263751, 0.48259346, 0.48249843, 0.48244741,
       0.48198507, 0.48198507, 0.48197649, 0.48155807, 0.48125649,
       0.48125649, 0.48044891, 0.47942668, 0.47942668, 0.47928011,
       0.47928011, 0.47796733, 0.47794206, 0.47794206, 0.47730766,
       0.47724351, 0.47710453, 0.47710453, 0.47595891, 0.47595891,
       0.47567261, 0.47567261, 0.47532701, 0.47526164, 0.47526164,
       0.47385647, 0.47370527, 0.47368668, 0.47355992, 0.47353517,
       0.47343796, 0.4731959 , 0.47317302, 0.47203343, 0.47188936,
       0.47172814, 0.47167922, 0.47135389, 0.47113535, 0.47106

In [53]:
np.array(vals[2])[np.argsort(vals[1], )][::-1][0:100]

array(['chr15:50261005:50262005', 'chr14:91138750:91139750',
       'chr15:76972340:76973340', 'chr15:76972340:76973340',
       'chr5:170278224:170279224', 'chr5:170278224:170279224',
       'chr19:4246203:4247203', 'chr3:170717723:170718723',
       'chr19:8831580:8832580', 'chr19:8831580:8832580',
       'chr20:31573271:31574271', 'chr20:31573271:31574271',
       'chr20:31573271:31574271', 'chrX:49878107:49879107',
       'chr2:197453455:197454455', 'chr19:8831398:8832398',
       'chr19:8831398:8832398', 'chr21:22209182:22210182',
       'chr21:22209182:22210182', 'chr13:52651567:52652567',
       'chr3:48996951:48997951', 'chr20:13785140:13786140',
       'chr20:13785140:13786140', 'chr20:13785140:13786140',
       'chr1:1796159:1797159', 'chr20:4081722:4082722',
       'chr3:170717493:170718493', 'chr11:2399340:2400340',
       'chr21:17689180:17690180', 'chr21:17689180:17690180',
       'chr11:72112578:72113578', 'chr11:72112578:72113578',
       'chr12:121742683:121743683', 'c